In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


#### We will use the Foursquare API to explore neighborhoods in the city of Boston. You will use the exploration function to get the most common categories of places in each neighborhood, and then use this function to group the neighborhoods into groups. You will use the k-means clustering algorithm to complete this task. Finally, you will use the Folium library to visualize the neighborhoods of the city of Boston and its emerging groups.

### Next, let's load the data.

In [13]:
df_boston=pd.read_csv("Boston_Neighbourhood.csv")
df_boston.head(26)

,OBJECTID,Name,Acres,Neighborhood_ID,SqMiles,ShapeSTArea,ShapeSTLength
0,27,Roslindale,1605.568237,15,2.51,6.993827e+07,53563.912597
1,28,Jamaica Plain,2519.245394,11,3.94,1.097379e+08,56349.937161
2,29,Mission Hill,350.853564,13,0.55,1.528312e+07,17918.724113
3,30,Longwood,188.611947,28,0.29,8.215904e+06,11908.757148
4,31,Bay Village,26.539839,33,0.04,1.156071e+06,4650.635493
5,32,Leather District,15.639908,27,0.02,6.812717e+05,3237.140537
6,33,Chinatown,76.324410,26,0.12,3.324678e+06,9736.590413
7,34,North End,126.910439,14,0.20,5.527506e+06,16177.826815
8,35,Roxbury,2108.469072,16,3.29,9.184455e+07,49488.800485
9,36,South End,471.535356,32,0.74,2.054000e+07,17912.333569


In [14]:
df_boston.shape

(26, 7)

#### Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [15]:
CLIENT_ID = 'FL33NARRRSLQVOWU5RS1S4GJ1PYCCU20MURF54X0LHOVGP10' # your Foursquare ID
CLIENT_SECRET = 'DYCI23FO5QM0GEYELTO102C4Y2R12KDJBYI5OWVL0OYF32XL' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FL33NARRRSLQVOWU5RS1S4GJ1PYCCU20MURF54X0LHOVGP10
CLIENT_SECRET:DYCI23FO5QM0GEYELTO102C4Y2R12KDJBYI5OWVL0OYF32XL


## Business Problem

### My project is to classify all the private schools that are located in the different neighborhoods of Boston. Through the Foursquare API I can get that data with its exact coordinates, centering the map in the chosen neighborhood. In this case we start with the Longwood neighborhood.

### The objective is to know which neighborhoods are left more depopulated with Private Schools and if it is necessary to build another school. And if so, where should we build it.

In [16]:
address = 'Longwood, Boston'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

42.3417184 -71.109922


### We use as a category in the query: Private school

In [17]:
search_query = 'private school'
radius = 500
print(search_query + ' .... OK!')

private school .... OK!


In [18]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FL33NARRRSLQVOWU5RS1S4GJ1PYCCU20MURF54X0LHOVGP10&client_secret=DYCI23FO5QM0GEYELTO102C4Y2R12KDJBYI5OWVL0OYF32XL&ll=42.3417184,-71.109922&v=20180604&query=private school&radius=500&limit=30'

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d9efa3ddb1d810039ee04d0'},
 'response': {'venues': [{'id': '4dcfcacc22718eed7a59cf99',
    'name': 'Winsor School Sports Fields',
    'location': {'lat': 42.33954939373447,
     'lng': -71.10750966712891,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.33954939373447,
       'lng': -71.10750966712891}],
     'distance': 312,
     'postalCode': '02215',
     'cc': 'US',
     'city': 'Boston',
     'state': 'MA',
     'country': 'United States',
     'formattedAddress': ['Boston, MA 02215', 'United States']},
    'categories': [{'id': '4f4528bc4b90abdf24c9de85',
      'name': 'Athletics & Sports',
      'pluralName': 'Athletics & Sports',
      'shortName': 'Athletics & Sports',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/sports_outdoors_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1570699837',
    'hasPerk': False},
   {'id': '5caf2a7cf00a70002c46e87b',
    'name': 'PrivateAborti

In [20]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4f4528bc4b90abdf24c9de85', 'name': 'A...",False,4dcfcacc22718eed7a59cf99,NaN,US,Boston,United States,NaN,312,"[Boston, MA 02215, United States]","[{'label': 'display', 'lat': 42.33954939373447...",42.339549,-71.107510,02215,MA,Winsor School Sports Fields,v-1570699837,NaN
1,"[{'id': '4bf58dd8d48988d177941735', 'name': 'D...",False,5caf2a7cf00a70002c46e87b,1180 Beacon St,US,Brookline,United States,NaN,486,"[1180 Beacon St, Brookline, MA 02446, United S...","[{'label': 'display', 'lat': 42.34400228575448...",42.344002,-71.114963,02446,MA,PrivateAbortionClinicBoston,v-1570699837,NaN
2,"[{'id': '4bf58dd8d48988d13d941735', 'name': 'H...",False,4b7b6fb4f964a5208a632fe3,103 Pilgrim Rd,US,Boston,United States,NaN,249,"[103 Pilgrim Rd, Boston, MA 02215, United States]","[{'label': 'display', 'lat': 42.3411298592223,...",42.341130,-71.106998,02215,MA,The Winsor School,v-1570699837,NaN
3,"[{'id': '4bf58dd8d48988d1e4941735', 'name': 'C...",False,5ceae1563e6741002c671acc,102 Pilgrim Rd,US,Boston,United States,NaN,284,"[102 Pilgrim Rd, Boston, MA 02215, United States]","[{'label': 'display', 'lat': 42.34070017, 'lng...",42.340700,-71.106749,02215,MA,Steve & Kate's Camp at The Winsor School,v-1570699837,553040846
4,"[{'id': '4e39a9cebd410d7aed40cbc4', 'name': 'C...",False,4d5aa0db8a7aba7aa107b70d,371-409 Brookline Ave,US,Boston,United States,Longwood Ave,370,"[371-409 Brookline Ave (Longwood Ave), Boston,...","[{'label': 'display', 'lat': 42.33922578901802...",42.339226,-71.106935,02115,MA,Windsor School Tennis Courts,v-1570699837,NaN


### We create a normalized dataframe with the results obtained in the json

In [35]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Winsor School Sports Fields,Athletics & Sports,NaN,US,Boston,United States,NaN,312,"[Boston, MA 02215, United States]","[{'label': 'display', 'lat': 42.33954939373447...",42.339549,-71.107510,02215,MA,4dcfcacc22718eed7a59cf99
1,PrivateAbortionClinicBoston,Doctor's Office,1180 Beacon St,US,Brookline,United States,NaN,486,"[1180 Beacon St, Brookline, MA 02446, United S...","[{'label': 'display', 'lat': 42.34400228575448...",42.344002,-71.114963,02446,MA,5caf2a7cf00a70002c46e87b
2,The Winsor School,High School,103 Pilgrim Rd,US,Boston,United States,NaN,249,"[103 Pilgrim Rd, Boston, MA 02215, United States]","[{'label': 'display', 'lat': 42.3411298592223,...",42.341130,-71.106998,02215,MA,4b7b6fb4f964a5208a632fe3
3,Steve & Kate's Camp at The Winsor School,Campground,102 Pilgrim Rd,US,Boston,United States,NaN,284,"[102 Pilgrim Rd, Boston, MA 02215, United States]","[{'label': 'display', 'lat': 42.34070017, 'lng...",42.340700,-71.106749,02215,MA,5ceae1563e6741002c671acc
4,Windsor School Tennis Courts,College Tennis Court,371-409 Brookline Ave,US,Boston,United States,Longwood Ave,370,"[371-409 Brookline Ave (Longwood Ave), Boston,...","[{'label': 'display', 'lat': 42.33922578901802...",42.339226,-71.106935,02115,MA,4d5aa0db8a7aba7aa107b70d
5,Lawrence School Tennis Courts,Park,Prince,US,Brookline,United States,Kent,408,"[Prince (Kent), Brookline, MA, United States]","[{'label': 'display', 'lat': 42.33996106623800...",42.339961,-71.114287,NaN,MA,4c0a8458009a0f47ec80e9bf
6,Lawrence School,Elementary School,NaN,US,Brookline,United States,NaN,566,"[Brookline, MA, United States]","[{'label': 'display', 'lat': 42.33972322305180...",42.339723,-71.116254,NaN,MA,4bd462a0a8b3a593d5a66b5f
7,Harvard School of Dental Medicine (HSDM),Medical School,55 Shattuck St,US,Boston,United States,NaN,590,"[55 Shattuck St, Boston, MA 02115, United States]","[{'label': 'display', 'lat': 42.33697427519438...",42.336974,-71.106703,02115,MA,4c6fdccd1f58199cb875367c
8,"Church Lab, Harvard Medical School",College Lab,NaN,US,Boston,United States,NaN,576,"[Boston, MA 02115, United States]","[{'label': 'display', 'lat': 42.33894140176183...",42.338941,-71.104005,02115,MA,4bfea2eddaf9c9b633fef8ef


In [36]:
dataframe_filtered.name

0                 Winsor School Sports Fields
1                 PrivateAbortionClinicBoston
2                           The Winsor School
3    Steve & Kate's Camp at The Winsor School
4                Windsor School Tennis Courts
5               Lawrence School Tennis Courts
6                             Lawrence School
7    Harvard School of Dental Medicine (HSDM)
8          Church Lab, Harvard Medical School
Name: name, dtype: object

### let's visualizat o Longwood the Private School in it.

In [38]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Longwood

# add a red circle marker to represent the Longwood
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup="Longwood",
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers

for lat, lng, name in zip(dataframe_filtered["lat"], dataframe_filtered["lng"], dataframe_filtered["name"]):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6,
        parse_html=False).add_to(venues_map)
# display map

venues_map